<a href="https://colab.research.google.com/github/cristianmejia00/clustering/blob/main/Topic_Models_using_BERTopic_TOPIC_MODEL_20241101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modeling with BERTopic

🔴 copied from the [Kubota Colab](https://colab.research.google.com/drive/1YsDp5_qGXGJKsEXsS8DO8CA_lqZc6EpA).  

`Topic Models` are methods to automatically organize a corpus of text into topics.

Topic Model process:
1. Data preparation
2. Tranform text to numeric vectors
3. Multidimensionality reduction
4. Clustering
5. Topic analysis
6. Cluster assignation


This notebook uses the library `BERTopic` which is a one-stop solution for topic modeling including handy functions for plotting and analysis. However, BERTopic does not have a function to extract the X and Y coords from UMAP. If we need the coordinates then use the notebooks `Topic_Models_using_Transformers` instead. In any other situation, when a quick analysis is needed this notebook may be better.

This notebook is also the one needed for the heatmap codes included in this folder.

`BERTopic` is Python library that handles steps 2 to 6.
BERT topic models use the transformer architechture to generate the embeds (i.e. the vector or numeric representation of words) and are currently the state-of-the-art method for vectorization.

This notebook shows how to use it.

---
Reading:
[Topic Modeling with Deep Learning Using Python BERTopic](https://medium.com/grabngoinfo/topic-modeling-with-deep-learning-using-python-bertopic-cf91f5676504)
[Advanced Topic Modeling with BERTopic](https://www.pinecone.io/learn/bertopic/)


# Requirements

## Packages installation and initialization

In [ ]:
#!pip install bertopic[visualization]

In [2]:
import pandas as pd
import numpy as np
import time
import math
import uuid
import re
import os
import json
import pickle
from datetime import date
from itertools import compress
from bertopic import BERTopic
from umap import UMAP
#from gensim.parsing.preprocessing import remove_stopwords
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer

## Connect your Google Drive

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
def find_e_keys(dictionary):
    # List comprehension to find keys starting with 'e'
    e_keys = [key for key in dictionary if str(key).lower().startswith('e')]
    return e_keys

# 🔴 Input files and options

Go to your Google Drive and create a folder in the root directory. We are going to save all related data in that directory.
Upload the dataset of news into the above folder.
- The dataset should be a `.csv` file.
- Every row in the dataset is a document
- It can any kind of columns. Some columns must contain the text we want to analyze. For example, a dataset of academic articles may contain a "Title" and/or "Abstract" column.

In [4]:
# The bibliometrics folder
# Colab
ROOT_FOLDER_PATH = "drive/MyDrive/Bibliometrics_Drive"

# Mac
ROOT_FOLDER_PATH = "/Users/cristian/Library/CloudStorage/GoogleDrive-cristianmejia00@gmail.com/My Drive/Bibliometrics_Drive"

# Change to the name of the folder where the dataset is uploaded inside the above folder
project_folder = 'Q338_synthetic_biology_test'

analysis_id = 'a01_tm__f01_e01__hdbs'

# Filtered label
settings_directive = "settings_analysis_directive_2025-08-04-10-59.json"

In [5]:
# Read settings
with open(f'{ROOT_FOLDER_PATH}/{project_folder}/{analysis_id}/{settings_directive}', 'r') as file:
    settings = json.load(file)

In [6]:
# Input dataset
dataset_file_path = f"{ROOT_FOLDER_PATH}/{settings['metadata']['project_folder']}/{settings['metadata']['filtered_folder']}/dataset_raw_cleaned.csv"

In [7]:
# Function to save files
def save_as_csv(df, save_name_without_extension, with_index):
    "usage: `save_as_csv(dataframe, 'filename')`"
    df.to_csv(f"{ROOT_FOLDER_PATH}/{save_name_without_extension}.csv", index=with_index)
    print("===\nSaved: ", f"{ROOT_FOLDER_PATH}/{save_name_without_extension}.csv")

In [8]:
# prompt: a function to save object to a pickle file
def save_object_as_pickle(obj, filename):
  """
  Saves an object as a pickle file.

  Args:
      obj: The object to be saved.
      filename: The filename of the pickle file.
  """
  with open(filename, "wb") as f:
    pickle.dump(obj, f)


In [9]:
# prompt: a function to load pickle object given a path
def load_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)


In [10]:
# Open the data file
df = pd.read_csv(f"{dataset_file_path}", encoding='latin-1')
print(df.shape)
df.head()

(17924, 42)


/var/folders/fj/y97jpw3n765_z56_jv9vqpgc0000gn/T/ipykernel_23828/2630543451.py:2: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{dataset_file_path}", encoding='latin-1')


,X_N,uuid,PT,AU,AF,TI,SO,LA,DT,DE,...,AR,DI,PG,WC,SC,OA,UT,Countries,IsoCountries,Institutions
0,1,fddb5139-d514-474c-a12a-5d87fb90e4fd,J,"Trump, BD; Galaitsi, SE; Appleton, E; Bleijs, ...","Trump, Benjamin D.; Galaitsi, S. E.; Appleton,...",Building biosecurity for synthetic biology,MOLECULAR SYSTEMS BIOLOGY,English,Editorial Material,NaN,...,e9723,10.15252/msb.20209723,6,Biochemistry & Molecular Biology,Biochemistry & Molecular Biology,"Green Published, gold",WOS:000557477800014,usa; netherlands; switzerland; italy; united k...,USA; NLD; CHE; ITA; GBR; FRA; BGR,us army; harvard univ; netherlands natl inst p...
1,2,8f3c324d-48e2-46c8-ab64-79b55f9e800b,J,"Takano, E; Bovenberg, RAL; Breitling, R","Takano, Eriko; Bovenberg, Roel A. L.; Breitlin...",A turning point for natural product discovery ...,MOLECULAR MICROBIOLOGY,English,Article,NaN,...,NaN,10.1111/j.1365-2958.2012.07984.x,10,Biochemistry & Molecular Biology; Microbiology,Biochemistry & Molecular Biology; Microbiology,Green Published,WOS:000300688800002,netherlands; united kingdom,NLD; GBR,univ groningen; dsm biotechnol ctr; univ glasgow
2,3,a504a2e3-38bb-46c2-aad7-34f8a0628ac8,J,"Arun, KB; Anoopkumar, AN; Sindhu, R; Binod, P;...","Arun, K. B.; Anoopkumar, A. N.; Sindhu, Raveen...",Synthetic biology for sustainable food ingredi...,SYSTEMS MICROBIOLOGY AND BIOMANUFACTURING,English,Review,Synthetics biology; Cell factory; Metabolic en...,...,NaN,10.1007/s43393-022-00150-3,13,Biotechnology & Applied Microbiology,Biotechnology & Applied Microbiology,NaN,WOS:001457594000002,india; peoples r china,IND; CHN,rajiv gandhi ctr biotechnol; christ; univ cali...
3,4,fb88babb-a79c-4fd0-9c74-cc53466ced07,J,"McLaughlin, JA; Pocock, M; Misirli, G; Madsen,...","McLaughlin, James Alastair; Pocock, Matthew; M...",VisBOL: Web-Based Tools for Synthetic Biology ...,ACS SYNTHETIC BIOLOGY,English,Article; Proceedings Paper,synthetic biology; visualization; Synthetic Bi...,...,NaN,10.1021/acssynbio.5b00244,3,Biochemical Research Methods,Biochemistry & Molecular Biology,NaN,WOS:000381847300009,united kingdom,GBR,newcastle univ; turing ate my hamster ltd
4,5,86021006-d6f5-448a-b0bf-7b1541b745a6,J,"Tan, X; Letendre, JH; Collins, JJ; Wong, WW","Tan, Xiao; Letendre, Justin H.; Collins, James...",Synthetic biology in the clinic: engineering v...,CELL,English,Review,NaN,...,NaN,10.1016/j.cell.2021.01.017,18,Biochemistry & Molecular Biology; Cell Biology,Biochemistry & Molecular Biology; Cell Biology,"Green Accepted, Green Submitted",WOS:000629633400007,usa,USA,harvard univ; massachusetts gen hosp; harvard ...




---



## PART 2: Topic Model

In [11]:
# bibliometrics_folder
# project_folder
# project_name_suffix
# ROOT_FOLDER_PATH = f"drive/MyDrive/{bibliometrics_folder}"

#############################################################
# Embeddings folder
embeddings_folder_name = settings['tmo']['embeds_folder']

# Which column has the year of the documents?
my_year = settings['tmo']['year_column']

# Number of topics. Select the number of topics to extract.
# Choose 0, for automatic detection.
n_topics = settings['tmo']['n_topics']

# Minimum number of documents per topic
min_topic_size = settings['tmo']['min_topic_size']

In [12]:
# Get the embeddings back.
embeddings = load_pickle(f"{ROOT_FOLDER_PATH}/{project_folder}/{settings['metadata']['filtered_folder']}/{embeddings_folder_name}/embeddings.pck")
corpus =     pd.read_csv(f"{ROOT_FOLDER_PATH}/{project_folder}/{settings['metadata']['filtered_folder']}/{embeddings_folder_name}/corpus.csv").reset_index(drop=True)

In [13]:
# Combine embeddings
documents = corpus.text.to_list()

In [53]:
# corpus['uuid'] = [uuid.uuid4() for _ in range(len(corpus.index))]
# corpus['X_N'] = [i for i in range(1, len(corpus.index)+1)]

In [14]:
len(documents)

17924

In [15]:
#len(embeddings) == len(documents)
len(embeddings['embeddings']) == len(documents)

True

In [16]:
from hdbscan.hdbscan_ import HDBSCAN
# Execute the topic model.
# I suggest changing the values marked with #<---
# The others are the default values and they'll work fine in most cases.
# This will take several minutes to finish.

# Initiate UMAP
umap_model = UMAP(n_neighbors=15,
                  n_components=5,
                  min_dist=0.0,
                  metric='cosine',
                  random_state=100)

if n_topics == 0:
  # Initiate topic model with HDBScan (Automatic topic selection)
  topic_model_params = HDBSCAN(min_cluster_size=min_topic_size,
                               metric='euclidean',
                               cluster_selection_method='eom',
                               prediction_data=True)
else:
  # Initiate topic model with K-means (Manual topic selection)
  topic_model_params = KMeans(n_clusters = n_topics)

# Initiate BERTopic
topic_model = BERTopic(umap_model = umap_model,
                       hdbscan_model = topic_model_params,
                       min_topic_size=min_topic_size,
                       #nr_topics=15,          #<--- Footnote 1
                       n_gram_range=(1,3),
                       language='english',
                       calculate_probabilities=True,
                       verbose=True)



# Footnote 1: This controls the number of topics we want AFTER clustering.
# Add a hashtag at the beggining to use the number of topics returned by the topic model.
# When using HDBScan nr_topics will be obtained after orphans removal, and there is no warranty that `nr_topics < HDBScan topics`.
# thus, with HDBScan `nr_topics` means N topics OR LESS.
# When using KMeans nr_topics can be used to further reduce the number of topics.
# We use the topics as returned by the topic model. So we do not need to activate it here.

In [17]:
# Compute topic model
#topics, probabilities = topic_model.fit_transform(documents, embeddings)
topics, probabilities = topic_model.fit_transform(documents, embeddings['embeddings'])

2026-02-03 15:52:19,537 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2026-02-03 15:52:35,888 - BERTopic - Dimensionality - Completed ✓
2026-02-03 15:52:35,891 - BERTopic - Cluster - Start clustering the reduced embeddings
2026-02-03 15:54:51,575 - BERTopic - Cluster - Completed ✓
2026-02-03 15:54:51,600 - BERTopic - Representation - Fine-tuning topics using representation models.
2026-02-03 15:55:02,884 - BERTopic - Representation - Completed ✓


In [ ]:
topic_embeddings = topic_model.topic_embeddings_
print(len(topic_embeddings[0]))
print(len(topic_embeddings))

384

In [20]:
# Get the list of topics
# Topic = the topic number. From the largest topic.
#         "-1" is the generic topic. Genericr keywords are aggegrated here.
# Count = Documents assigned to this topic
# Name = Top 4 words of the topic based on probability
# Representation = The list of words representing this topic
# Representative_Docs = Documents assigned to this topic
tm_summary = topic_model.get_topic_info()
tm_summary

,Topic,Count,Name,Representation,Representative_Docs
0,-1,6838,-1_the_of_and_to,"[the, of, and, to, in, for, synthetic, that, w...",[Production of bulk chemicals via novel metabo...
1,0,230,0_light_optogenetic_optogenetics_control,"[light, optogenetic, optogenetics, control, ph...",[Optogenetic switches for lightcontrolled gene...
2,1,188,1_circuits_logic_circuit_genetic,"[circuits, logic, circuit, genetic, gates, com...",[Constructing artificial neural networks using...
3,2,170,2_cellfree_cfps_protein synthesis_cellfree pro...,"[cellfree, cfps, protein synthesis, cellfree p...",[Applications of cellfree protein synthesis in...
4,3,165,3_crisprcas_crispr_editing_cas,"[crisprcas, crispr, editing, cas, genome, geno...",[TALENsan indispensable tool in the era of CRI...
...,...,...,...,...,...
481,480,5,480_regenerative_regenerative medicine_ooc_stem,"[regenerative, regenerative medicine, ooc, ste...",[Enhanced MyoDInduced Transdifferentiation to ...
482,481,5,481_synthetic bacteria_algorithms_evolutionary...,"[synthetic bacteria, algorithms, evolutionary,...",[A Microbial Screening in Silico Method for th...
483,482,5,482_apoai_tryptophan_catdc_tmo,"[apoai, tryptophan, catdc, tmo, ohtrp, tryptop...",[Enhancement of tryptophan monooxygenase therm...
484,483,5,483_networks_competencies_functional networks_...,"[networks, competencies, functional networks, ...",[Computational modeling approaches for studyin...


In [18]:
# Save the topic model assets
tm_folder_path = f'{ROOT_FOLDER_PATH}/{project_folder}/{settings["metadata"]["analysis_id"]}'

if not os.path.exists(tm_folder_path):
  !mkdir $tm_folder_path

tm_summary.to_csv(f'{tm_folder_path}/topic_model_info.csv', index=False)

In [19]:
# Number of topics found
found_topics = max(tm_summary.Topic) + 1
found_topics

454

In [20]:
# Confirm all documents are assigned
sum(tm_summary.Count) == len(corpus)

True

In [21]:
# Get top 10 terms for a topic
topic_model.get_topic(0)

[('light', 0.010095946059032294),
 ('optogenetic', 0.008407853739494481),
 ('optogenetics', 0.005395388607795831),
 ('control', 0.004322076734668476),
 ('photoreceptors', 0.0037023576505819714),
 ('blue', 0.003335360115147941),
 ('control of', 0.0032093461443183863),
 ('gene expression', 0.0030234408045253826),
 ('expression', 0.002919226814608277),
 ('optogenetic tools', 0.002579580475332183)]

In [22]:
# Get the top 10 documents for a topic
topic_model.get_representative_docs(0)

['Photosensitive degron variants for tuning protein stability by light Background Regulated proteolysis by the proteasome is one of the fundamental mechanisms used in eukaryotic cells to control cellular behavior Efficient tools to regulate protein stability offer synthetic influence on molecular level on a selected biological process Optogenetic control of protein stability has been achieved with the photosensitive degron psd module This engineered tool consists of the photoreceptor domain light oxygen voltage  LOV from Arabidopsis thaliana phototropin fused to a sequence that induces direct proteasomal degradation which was derived from the carboxyterminal degron of murine ornithine decarboxylase The abundance of target proteins tagged with the psd module can be regulated by blue light if the degradation tag is exposed to the cytoplasm or the nucleus Results We used the model organism Saccharomyces cerevisiae to generate psd module variants with increased and decreased stabilities in

In [64]:
# Others

# # Get the number of documents per topic (same as in the table above)
# topic_model.get_topic_freq(0)

# # Get the main keywords per topic
# topic_model.get_topics()

In [23]:
# Print the parameters used. (For reporting)
topic_model.get_params()

{'calculate_probabilities': True,
 'ctfidf_model': ClassTfidfTransformer(),
 'embedding_model': None,
 'hdbscan_model': HDBSCAN(prediction_data=True),
 'language': 'english',
 'low_memory': False,
 'min_topic_size': 5,
 'n_gram_range': (1, 3),
 'nr_topics': None,
 'representation_model': None,
 'seed_topic_list': None,
 'top_n_words': 10,
 'umap_model': UMAP(angular_rp_forest=True, metric='cosine', min_dist=0.0, n_components=5, n_jobs=1, random_state=100, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True}),
 'vectorizer_model': CountVectorizer(ngram_range=(1, 3)),
 'verbose': True,
 'zeroshot_min_similarity': 0.7,
 'zeroshot_topic_list': None}

In [24]:
tm_params = dict(topic_model.get_params())
for key, value in tm_params.items():
    tm_params[key]=  str(value)
with open(f'{tm_folder_path}/topic_model_params.json', 'w') as f:
    json.dump(tm_params, f, ensure_ascii=False, indent=4)
    print('Done')

Done


In [25]:
# Get the topic score for each paper and its assigned topic
topic_distr, _ = topic_model.approximate_distribution(documents, batch_size=1000)
distributions = [distr[topic] if topic != -1 else 0 for topic, distr in zip(topics, topic_distr)]

100%|██████████| 18/18 [00:42<00:00,  2.37s/it]


In [26]:
topic_distr

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [39]:
# Document information. Including the topic assignation
dataset_clustering_results = topic_model.get_document_info(documents, df = corpus, metadata={"Score": distributions})

# Check for orphans (Topic == -1), save and remove them
if -1 in dataset_clustering_results['Topic'].values:
    orphans = dataset_clustering_results[dataset_clustering_results['Topic'] == -1]
    orphans.to_csv(f'{tm_folder_path}/orphans.csv', index=False)
    dataset_clustering_results = dataset_clustering_results[dataset_clustering_results['Topic'] != -1]

In [40]:
# Standar format for report analysis
dataset_clustering_results = dataset_clustering_results.reset_index(drop=True)
dataset_clustering_results = dataset_clustering_results.drop(columns=['text'])
dataset_clustering_results['X_E'] = dataset_clustering_results['Score']
dataset_clustering_results['X_C'] = dataset_clustering_results['Topic'] + 1



# Assign 'level0' based on cluster coverage (90%)
cluster_counts = dataset_clustering_results['X_C'].value_counts().sort_values(ascending=False)
total_rows = len(dataset_clustering_results)
cumulative = cluster_counts.cumsum() / total_rows
main_clusters = cluster_counts.index[cumulative <= 0.9].tolist()
dataset_clustering_results['level0'] = dataset_clustering_results['X_C'].apply(lambda x: x if x in main_clusters else 99999)
dataset_clustering_results.head()


,UT,uuid,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document,Score,X_E,X_C,level0
0,WOS:000557477800014,fddb5139-d514-474c-a12a-5d87fb90e4fd,Building biosecurity for synthetic biology The...,26,26_security_biosecurity_risks_risk,"[security, biosecurity, risks, risk, bioterror...",[Emerging Technologies Biosecurity and Consequ...,security - biosecurity - risks - risk - bioter...,0.084229,False,0.076335,0.076335,27,27
1,WOS:001457594000002,a504a2e3-38bb-46c2-aad7-34f8a0628ac8,Synthetic biology for sustainable food ingredi...,53,53_food_foods_meat_milk,"[food, foods, meat, milk, fermented, fermentat...",[Foods of the Future Challenges Opportunities ...,food - foods - meat - milk - fermented - ferme...,0.195523,False,0.846876,0.846876,54,54
2,WOS:000381847300009,fb88babb-a79c-4fd0-9c74-cc53466ced07,VisBOL WebBased Tools for Synthetic Biology De...,231,231_visual_sbol visual_sbol_glyphs,"[visual, sbol visual, sbol, glyphs, diagrams, ...",[Synthetic biology open language visual SBOL v...,visual - sbol visual - sbol - glyphs - diagram...,0.330637,False,0.479181,0.479181,232,232
3,WOS:000629633400007,86021006-d6f5-448a-b0bf-7b1541b745a6,Synthetic biology in the clinic engineering va...,267,267_leishmaniasis_leishmania_parasite_epitopes,"[leishmaniasis, leishmania, parasite, epitopes...",[Immunometabolic Circuits in Infection for Adv...,leishmaniasis - leishmania - parasite - epitop...,0.009358,False,0.000000,0.000000,268,268
4,WOS:000441180800005,2e190912-dadb-4e4f-9d94-8afa0b8a90dd,Synthetic biology of microbes synthesizing pol...,61,61_pha_phas_polyhydroxyalkanoates_phb,"[pha, phas, polyhydroxyalkanoates, phb, pha pr...",[Microbial synthesis of polyhydroxyalkanoate b...,pha - phas - polyhydroxyalkanoates - phb - pha...,0.022706,False,1.000000,1.000000,62,62


In [41]:
# Standar format for report analysis
dataset_clustering_results['cl99'] = [True if x == 99 else False for x in dataset_clustering_results['level0']]
dataset_clustering_results['cl-99'] = [True if x == 99 else False for x in dataset_clustering_results['level0']]
dataset_clustering_results.head()

,UT,uuid,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document,Score,X_E,X_C,level0,cl99,cl-99
0,WOS:000557477800014,fddb5139-d514-474c-a12a-5d87fb90e4fd,Building biosecurity for synthetic biology The...,26,26_security_biosecurity_risks_risk,"[security, biosecurity, risks, risk, bioterror...",[Emerging Technologies Biosecurity and Consequ...,security - biosecurity - risks - risk - bioter...,0.084229,False,0.076335,0.076335,27,27,False,False
1,WOS:001457594000002,a504a2e3-38bb-46c2-aad7-34f8a0628ac8,Synthetic biology for sustainable food ingredi...,53,53_food_foods_meat_milk,"[food, foods, meat, milk, fermented, fermentat...",[Foods of the Future Challenges Opportunities ...,food - foods - meat - milk - fermented - ferme...,0.195523,False,0.846876,0.846876,54,54,False,False
2,WOS:000381847300009,fb88babb-a79c-4fd0-9c74-cc53466ced07,VisBOL WebBased Tools for Synthetic Biology De...,231,231_visual_sbol visual_sbol_glyphs,"[visual, sbol visual, sbol, glyphs, diagrams, ...",[Synthetic biology open language visual SBOL v...,visual - sbol visual - sbol - glyphs - diagram...,0.330637,False,0.479181,0.479181,232,232,False,False
3,WOS:000629633400007,86021006-d6f5-448a-b0bf-7b1541b745a6,Synthetic biology in the clinic engineering va...,267,267_leishmaniasis_leishmania_parasite_epitopes,"[leishmaniasis, leishmania, parasite, epitopes...",[Immunometabolic Circuits in Infection for Adv...,leishmaniasis - leishmania - parasite - epitop...,0.009358,False,0.000000,0.000000,268,268,False,False
4,WOS:000441180800005,2e190912-dadb-4e4f-9d94-8afa0b8a90dd,Synthetic biology of microbes synthesizing pol...,61,61_pha_phas_polyhydroxyalkanoates_phb,"[pha, phas, polyhydroxyalkanoates, phb, pha pr...",[Microbial synthesis of polyhydroxyalkanoate b...,pha - phas - polyhydroxyalkanoates - phb - pha...,0.022706,False,1.000000,1.000000,62,62,False,False


In [42]:
dataset_clustering_results.level0.value_counts()

level0
99999    1116
1         233
2         179
3         169
4         162
         ... 
298        11
312        10
311        10
320        10
317        10
Name: count, Length: 303, dtype: int64

In [43]:
# Save the dataframe
dataset_clustering_results.to_csv(f'{tm_folder_path}/dataset_minimal.csv', index=False)

In [44]:
# Save the topic model
topic_model.save(f'{tm_folder_path}/topic_model_object.pck')

2025-08-04 12:19:39,891 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.




---

